# Import libraries

In [1]:
!pip install -qq datasets evaluate transformers[sentencepiece]
!pip install -qq accelerate
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

In [3]:
notebook_login()

# Load dataset
Raw data: https://github.com/fsoft-ailab/Poem-Generator

In [30]:
data_path = './poems_dataset.csv'
raw_df = pd.read_csv(data_path)
raw_df.head(5)

,id,content,title,url,genre
0,139248,ngày đông se sắt lạnh trong lòng\ncó việc đi n...,SAY NẮNG,https://www.facebook.com/groups/48640773509859...,7 chu
1,139249,ôm đàn thao thức đến nữa đêm\nréo rắt cung âm ...,NaN,https://www.facebook.com/groups/17645444269765...,7 chu
2,139250,tết có người vui có kẻ buồn\nngười cười toe to...,TẾT HAI THÁI CỰC,https://www.facebook.com/groups/17645444269765...,7 chu
3,139251,đã quá ba mươi mộng lỡ làng\nđi tìm day dứt mả...,TRÁI NGANG ĐỨC HẠNH,https://www.facebook.com/groups/48640773509859...,7 chu
4,139252,mai đào nở rộ đón nàng xuân\nsợi nắng hanh vàn...,DÁNG XUÂN,https://www.facebook.com/groups/17645444269765...,7 chu


In [31]:
raw_df = raw_df[['content', 'title', 'url', 'genre']]

# Preprocessing

Get 7-word poems for training

In [32]:
raw_df = raw_df[raw_df['genre']=='7 chu']

Get 2000 random poems for training

In [33]:
raw_df = raw_df.sample(n=600, random_state=42)

Some preprocessing function

In [34]:
def capitalize_first_letter_of_each_sentence(content):
    sentences = content.split('\n')
    sentences = [sentence.capitalize() for sentence in sentences]
    return '\n'.join(sentences)

def preprocess_content(content):
    samples = []
    poem_parts = content.split('\n\n')
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split('\n')
        samples.append(poem_in_lines)
    return samples

def preprocess_dataset(df):
    df = df.explode('content')
    df.reset_index(drop=True, inplace=True)
    df = df.dropna(subset=['content'])
    df['content'] = df['content'].apply(lambda x: '\n'.join(x))
    return df

raw_df['content'] = raw_df['content'].apply(capitalize_first_letter_of_each_sentence)
raw_df['content'] = raw_df['content'].apply(lambda x: preprocess_content(x))
cleaned_df = preprocess_dataset(raw_df)
cleaned_df.head(5)

,content,title,url,genre
0,Nhớ tiếng kinh cầu huế mộng mơ\nSông hương nướ...,NaN,https://poem.tkaraoke.com/100080/hue_tinh_yeu_...,7 chu
1,Giao mùa thổn thức giọt buồn mơ\nVẳng giữa trờ...,NaN,https://poem.tkaraoke.com/100080/hue_tinh_yeu_...,7 chu
2,Thân thẳng tư niên đứng dưới trời\nNhìn dòng t...,NaN,https://poem.tkaraoke.com/90939/vinh_cay_truc....,7 chu
3,Xuân mới đã sang khắp chốn rồi\nĐào mai khoe s...,MỪNG XUÂN MỚI,https://www.facebook.com/groups/31960706210236...,7 chu
4,Ai nấy rạng ngời từng khuôn mặt\nNăm mới an là...,MỪNG XUÂN MỚI,https://www.facebook.com/groups/31960706210236...,7 chu


In [35]:
cleaned_df.to_csv('./cleaned_poem_data.csv', index=True)

# Preprare dataset for training

Create Dataset

In [36]:
dataset = Dataset.from_pandas(cleaned_df)
dataset

Dataset({
    features: ['content', 'title', 'url', 'genre'],
    num_rows: 1563
})

In [37]:
TEST_SIZE = 0.1
dataset = dataset.train_test_split(test_size=TEST_SIZE)
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'title', 'url', 'genre'],
        num_rows: 1406
    })
    test: Dataset({
        features: ['content', 'title', 'url', 'genre'],
        num_rows: 157
    })
})

Tokenization

In [38]:
# https://huggingface.co/danghuy1999/gpt2-viwiki
tokenizer = GPT2Tokenizer.from_pretrained('danghuy1999/gpt2-viwiki', clean_up_tokenization_spaces=True)

In [39]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 200

def preprocess_function(row):
    return tokenizer(
        row['content'],
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

tokenized_poem_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset['train'].column_names,
)

Map (num_proc=4):   0%|          | 0/1406 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/157 [00:00<?, ? examples/s]

Test tokenizer

In [40]:
tokenizer('Chuẩn bị tokenizer hiệu quả', max_length=10, padding='max_length', truncation=True)

{'input_ids': [21198, 469, 1189, 6745, 12378, 865, 1013, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}

In [41]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Finetuning GPT2

Load model

In [42]:
model = GPT2LMHeadModel.from_pretrained('danghuy1999/gpt2-viwiki').to('cuda')

In [43]:
# Config training arguements
training_args = TrainingArguments(
    output_dir='gpt2_viet_tho_7_chu',
    save_strategy='epoch',
    learning_rate=5e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True
)

Train and push model to huggingface hub

In [44]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset['train'],
    eval_dataset=tokenized_poem_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-44-4059e61a2066>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,6.427600
1000,5.725400
1500,5.437700


TrainOutput(global_step=1760, training_loss=5.784055813876066, metrics={'train_runtime': 671.196, 'train_samples_per_second': 20.948, 'train_steps_per_second': 2.622, 'total_flos': 1435064832000000.0, 'train_loss': 5.784055813876066, 'epoch': 10.0})

In [45]:
trainer.push_to_hub(commit_message="Training completed")

events.out.tfevents.1736522303.7edeeed54b53.276.1:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/loc1105/gpt2_viet_tho_7_chu/commit/b2f02948175523e6cdea7bdfecc669c4c63effde', commit_message='Training completed', commit_description='', oid='b2f02948175523e6cdea7bdfecc669c4c63effde', pr_url=None, repo_url=RepoUrl('https://huggingface.co/loc1105/gpt2_viet_tho_7_chu', endpoint='https://huggingface.co', repo_type='model', repo_id='loc1105/gpt2_viet_tho_7_chu'), pr_revision=None, pr_num=None)

# Inference

In [54]:
from transformers import pipeline

prompt = 'Trong buổi chiều mưa buồn vắng lặng'
generator = pipeline('text-generation', model='loc1105/gpt2_viet_tho_7_chu')

results = generator(
    prompt,
    max_new_tokens=100,
    do_sample=True,
    top_k=40,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2,
)
results = results[0]['generated_text']

# Cắt kết quả thành các dòng 7 chữ
max_words_per_line = 10
min_words_per_line = 5  # Giới hạn dòng tối thiểu có 5 từ
lines = results.split('\n')

for line in lines:
    words = line.split()
    if len(words) > max_words_per_line:
        words = words[:max_words_per_line]
    if len(words) >= min_words_per_line:  # Kiểm tra dòng có ít nhất 5 từ
        print(" ".join(words))


Device set to use cuda:0


Trong buổi chiều mưa buồn vắng lặng
Thả ai say đắm trong lòng nhớ
Khắc khoải tình em vẫn đang dang dở
Ngõn thức nỗi đau những đêm đầu
Để ta lên sông dài dòng chảy
Luyện mờ mộng đâu còn mãi nữa
Sao mà nghe lời ấy không nói
Ta về rồi lại thấy mình tôi
Đêm xưa đã hết nhưng vậy thôi
Hợi trăng mơ hồ như cành xanh
Gối tay trông đợi thu đến tận đông
Rực rỡ ánh sao có màu nâu


In [56]:
from transformers import pipeline

prompt = 'Em như hoa nở giữa trời xuân'

results = generator(
    prompt,
    max_new_tokens=100,
    do_sample=True,
    top_k=40,
    top_p=0.95,
    temperature=1.2,
    repetition_penalty=1.2,
)
results = results[0]['generated_text']

# Cắt kết quả thành các dòng 7 chữ
max_words_per_line = 10
min_words_per_line = 5  # Giới hạn dòng tối thiểu có 5 từ
lines = results.split('\n')

for line in lines:
    words = line.split()
    if len(words) > max_words_per_line:
        words = words[:max_words_per_line]
    if len(words) >= min_words_per_line:  # Kiểm tra dòng có ít nhất 5 từ
        print(" ".join(words))


Em như hoa nở giữa trời xuân
Sao chẳng hề quen đâu anh biết chuyện
Còn bên trái mấy phương diện tươi lạ
Đò đêm qua sông dài vắng đầy vơi
Vành ngàn giá hồn thương nhớ trông chờ đợi
Một hôm trời đang ở kinh thành
Bên gió mây bay trong lòng biển
Để em những dịp vui khó ai thì vội vã
Thênh thang lại mãi tim nắng ấm ấm
Hãy đó không biết người đến tối thu
Nhỡ giấc thấy mưa đã vượt trước đồi
